# Summarization text for healthcare Fintuning Flan-t5

In [ ]:
# !pip install -q openpyxl xlrd
# #!pip install -q datasets transformers==4.28.0 rouge-score nltk sentencepiece
# !pip install -q datasets transformers rouge-score nltk sentencepiece
# !pip install -q torch

In [ ]:
!pip -q install transformers==4.28.0 datasets==2.12.0 sagemaker==2.156.0 --upgrade

In [ ]:
import torch
import datasets
from datasets import Dataset
from datasets import load_metric
from datasets import concatenate_datasets
from datasets.filesystems import S3FileSystem

import transformers
from transformers import AutoTokenizer

import sagemaker
from sagemaker.huggingface import HuggingFace

sess = sagemaker.Session()

In [ ]:
import pandas as pd
# dataset from https://github.com/abachaa/MeQSum

df = pd.read_excel('MeQSum_ACL2019_BenAbacha_Demner-Fushman.xlsx')
df = df.drop('File', axis=1)
df = df.rename(columns={'CHQ':'Text'})
df = df.dropna()
df['Text']= df['Text'].apply(lambda x: x.lower())
df['Summary'] = df['Summary'].apply(lambda x: x.lower())
df['Id'] = range(0, len(df.index))
df = df[['Id', 'Text', 'Summary']]
df = df.sample(frac=1).reset_index(drop=True)
df

In [ ]:
model_checkpoint = 'google/flan-t5-small'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
train = df[:700]
val = df[700:900]
test = df[900:]
print('train: {}, val: {}, test: {}'.format(train.shape, val.shape, test.shape))

In [ ]:
# Dataset from dataframe
train_dataset = Dataset.from_pandas(train)
val_dataset = Dataset.from_pandas(val)
test_dataset = Dataset.from_pandas(test)

In [ ]:
# max input lengith and max target length based on dataset.
tokenized_inputs = concatenate_datasets([train_dataset, val_dataset, test_dataset]).map(lambda x: tokenizer(x["Text"], truncation=True), batched=True, remove_columns=["Text", "Summary"])
max_input_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max input length: {max_input_length}")

tokenized_targets = concatenate_datasets([train_dataset, val_dataset, test_dataset]).map(lambda x: tokenizer(x["Summary"], truncation=True), batched=True, remove_columns=["Text", "Summary"])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

In [ ]:
# summarizing template
def preprocess_function(sample,padding="max_length"):
    inputs = ["summarize: " + item for item in sample["Text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, padding=padding, truncation=True)

    labels = tokenizer(text_target=sample["Summary"], max_length=max_target_length, padding=padding, truncation=True)

    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
# Tokenized dataset
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)

print(f"Keys of tokenized dataset: {tokenized_train.features}")

In [ ]:
# Uploading dataset to S3
s3 = S3FileSystem()

bucket = sess.default_bucket()
s3_prefix = "huggingface/meqsum-flan-t5-summarization"

dataset_input_path = "s3://{}/{}".format(bucket, s3_prefix)
train_input_path = "{}/train".format(dataset_input_path)
valid_input_path = "{}/validation".format(dataset_input_path)

print(dataset_input_path)
print(train_input_path)
print(valid_input_path)

tokenized_train.save_to_disk(train_input_path, fs=s3)
tokenized_val.save_to_disk(valid_input_path, fs=s3)

In [ ]:
# hyperparameters
hyperparameters = {
    "epochs": 10,
    "learning-rate": 2e-5,
    "train-batch-size": 4,
    "eval-batch-size": 4,
    "model-name": model_checkpoint,
}

In [ ]:
metric_definitions=[
    {'Name': 'loss', 'Regex': "'loss': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'learning_rate', 'Regex': "'learning_rate': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_loss', 'Regex': "'eval_loss': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'epoch', 'Regex': "'epoch': ([0-9]+(.|e\-)[0-9]+),?"}
]

In [ ]:
huggingface_estimator = HuggingFace(
    role=sagemaker.get_execution_role(),
    entry_point="train.py",
    dependencies=["requirements.txt"],
    hyperparameters=hyperparameters,
    transformers_version="4.26.0",
    pytorch_version="1.13.1",
    py_version="py39",
    instance_type="ml.p3.2xlarge",
    instance_count=1,
    metric_definitions=metric_definitions
    # distribution={"smdistributed": {"dataparallel": {"enabled": True}}}, # For distributed training.
)

In [ ]:
huggingface_estimator.fit({"train": train_input_path, "valid": valid_input_path})

In [ ]:
huggingface_estimator.model_data

In [ ]:
huggingface_predictor = huggingface_estimator.deploy(
    initial_instance_count=1, instance_type="ml.p3.2xlarge"
)

In [ ]:
test_dataset[0]['Text']

In [ ]:
test_data = {"inputs": f"summarize: {test_dataset[0]['Text']}"}
test_data

In [ ]:
prediction = huggingface_predictor.predict(test_data)
print(prediction)

In [ ]:
test_dataset[0]['Summary']

## Clean up

In [ ]:
huggingface_predictor.delete_model()
huggingface_predictor.delete_endpoint()